## PSEUDOCODE STRUCTURE

1. API Call #1 to get all tournament IDs (or at least the top like 1000)
2. API Call #2 to get all the data for each tournament ID we scraped in step 1
3. Line graph to show year by year total earnings potential in these tournaments (maybe by month/quarter if available)
4. Animate Line graph
5. LDA to generate fake tournament names for fun

In [2]:
import requests
import pandas as pd
import json
import time

In [3]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
esportsarr = []
tourniterator = 0

for n in range(50):
    data = requests.get(f"https://api.esportsearnings.com/v0/LookupTournamentById?apikey=6fbda7c3f7cb1d84286157bdb8f3defc9433ac28eae09a95282cd2808dea2ec6&tournamentid={tourniterator}"
                      , verify=False)
    esportsarr.append(data.text)
    tourniterator+=1 
    with open('data.json', 'a') as outfile:
        json.dump(esportsarr, outfile)
        time.sleep(1) #api calls must be 1 second apart

In [4]:
#this cell will call the API and gather all of the direct tournament data in a JSON file
tidjson = []
offsetiter = 0

for n in range (2):
    tids = requests.get(f"http://api.esportsearnings.com/v0/LookupRecentTournaments?apikey=6fbda7c3f7cb1d84286157bdb8f3defc9433ac28eae09a95282cd2808dea2ec6&offset={offsetiter}"
                        , verify = False)
    tidjson.append(tids.text)
    offsetiter+=100
    with open('tids.json', 'a') as outfile:
        json.dump(tidjson, outfile)
        time.sleep(1)


In [ ]:
#this cell should take the JSON as an input 
df = pd.read_json('filepathtojson')

#with open('temp.txt') as fh:
#    df = pd.DataFrame(json.load(fh)['Default'])  this might have to be the way you load in the json file...

In [ ]:
#drop all duplicate IDs from the json file

In [ ]:
#this cell should iterate over the list of tourney keys and compile them in a dataframe (might have to be saved if too large)
datastore = []

for key in df['keys']:
    data = requests.get(f"https://api.esportsearnings.com/v0/LookupTournamentById? \
    apikey=6fbda7c3f7cb1d84286157bdb8f3defc9433ac28eae09a95282cd2808dea2ec6&tournamentid={key}")
    datastore.append(data.text)
    

In [ ]:
#this cell should visualize the time series of esports earnings
import chart_studio.plotly as py
import matplotlib as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go

plotly.offline.init_notebook_mode() #this sets plotly to offline mode (aka no cloud comms)

In [ ]:
#maybe create another visualization that shows where the number of tournaments are happening?? 
#do a separate color'd bar on stacked bar for single vs team play???
#wait change this to a map....
fig = go.Figure(data = [ 
    go.Bar(name = 'Solo Tournies', x = datastore['state'], y = datastore['state'].value_counts()) 
    go.Bar(name = 'Teamplay', x = datastore['state'], y = datastore['state'].value_counts())
           ]
)
           
fig.update_layout(barmode='stack')

fig.show()

In [ ]:
earnings = go.Scatter( #this call is sorta pseudocode
        x = datastore['earnings'].groupby(datastore['Year']),
        y = datastore['Year'],
        mode='lines',
        name='ESports Earnings by Year')
    
chartdata = go.Data([earnings])

plotly.offline.iplot(data, filename = 'basic-line')

In [ ]:
fig = go.Figure() #figure out which one works... one of these is outdated

fig.add_trace(go.Scatter(
            x = datastore['earnings'].groupby(datastore['Year']),
            y = datastore['Year'],
            mode='lines', 
            name='Esports Earnings by Year'))

fig.show()

In [ ]:
#this cell should call API for highest earning players (is one of the API modes)
bigmoney = []

bigmoneydata = requests.get("https://api.esportsearnings.com/v0/LookupHighestEarningPlayerByGam?\
    apikey=6fbda7c3f7cb1d84286157bdb8f3defc9433ac28eae09a95282cd2808dea2ec6")
bigmoney.append(bigmoneydata.text)



In [ ]:
#this cell should visualize the highest earning players, maybe top 5?
figmoney = go.Figure() #figure out which one works... one of these is outdated

figmoney.add_trace(go.Bar(
            x = datastore['playernameusdearnings'],
            y = datastore['usdearnings'],
            name='Highest Total Player Earnings'))

figmoney.show()

In [ ]:
#this cell for shiggles should use LDA to generate some fake tournament titles